This notebook aims to create extract key words from lithology descriptions. The notebook will work on building a dictionary for key geological and textural words. This is done by a combination of defining synonymous words explicitly (e.g. gravel and pebbles) and fuzzy string matching to pickup spelling errors and shorthand (e.g. s/stn vs sandstone). For now we are focussing on single words but future endeavors will work on sequences of consecutive words (i.e. well-rounded, red-brown)

One a significant portion of words within the lithology logs are standardised we will attempt an automatic classification of intervals into a set number of key lithologies using a bag of words approach

Neil Symington
neil.symington@ga.gov.au

In [139]:
import pandas as pd
import json
import numpy as np
import re
from collections import Counter
from fuzzywuzzy import process
import time

In [128]:
# Define key functions

def add_to_dictionary(value, key, dictionary):
    """
    A function for appending an object to a dictionary
    entry
    
    @param value: object to be added to the dictionary
    @param key: dictionary key
    @dictionary: dictionary
    returns
    dictionary with added values
    """
    # Check if it already exists if so add nothing
    try:
        if values in dictionary[key]:
            return dictionary
        # Otherwise add the word
        else:
            dictionary[key] += [values]
            return dictionary
    except KeyError:
        dictionary[key] = [values]
        return dictionary

def search(kword_dict, word):
    """
    A search function for finding if a keword
    exists as a list element within a dictionary entry
    
    @param kword_dict: dictionary with keywords
    @word: string
    returns
    the key for the entry if it exists or None if not
    """
    word = word.lower()
    for k in kword_dict:
        for v in kword_dict[k]:
            if word in v:
                return k
    return None

def findUnregisteredWords(kword_dict, lithDescription):
    """
    A function for searching through a sequence of 
    lithology desciptions, removing punctuation and cases
    and then returning all the words that are not contained
    as values within the entries of a key word dictoinary
    
    @param kword_dict: key-word dictionary
    @param lithDescription: sequence of strings describing
    borehole lithology
    
    returns
    list of unregstered words
    
    """
    # Create list into which unregistered words will be appended
    unregistered_words = []
    # Iterate through lithology descriptions
    for item in lithDescription:
        # This is to prevent errors if there are None elements
        # in the list
        if item is not None:
        # remove punctuation and split into list
            L = re.sub(r'[^\w\s]','', item).split()
            # iterte through word list
            for word in L:
                # If the word does not exist yet in dictionary
                # Append it to the list
                if search(kword_dict, word) is None:
                    
                    unregistered_words.append(word.lower())
    return unregistered_words

def bag_of_words(string, kwords):
    """
    Function for counting the occurene of key words
    within a lithology description
    
    @param string: string from which to search for keywords
    @parma kwords: dictionary of keywords where the key is the
    keyword and the entries are a list of 'synonyms'
    returns
    
    dictionary with score denoting the number of times a
    key word was seen
    """
    # remove punctuation and split into list
    L = re.sub(r'[^\w\s]','', string).split()
    # mak entirely lowercase
    L = [x.lower() for x in L]
    
    newDict = {}
    # Iterate through kewords
    for item in kwords:
        # Assign the keyword a zero meaning unseen
        # This will be updated if the word is 
        # found in the lithology description
        newDict[item] = 0
        # ITerate through mapped words
        for entry in kwords[item]:
            # if the entry is in the string
            # assign the item a 1 in newDict
            if entry in L:
                newDict[item] += 1
    
    # Return the dictionary
    return newDict

In [84]:
# Create a dicitonary with a set of keywords for interpreting drilling log lithology and texture descriptions

lithology_kwords = {}

kword_file = r"C:\Users\symin\OneDrive\Documents\GA\lithology_classification\ngis_borehole_keyword_search.csv"

df_keywords = pd.read_csv(kword_file)

for index, row in df_keywords.iterrows():
    kword = row['key_words']
    lithology_kwords[kword] = [kword]

print(lithology_kwords)

{'bioturbation': ['bioturbation'], 'breccia': ['breccia'], 'cement': ['cement'], 'clast': ['clast'], 'cross-bedding': ['cross-bedding'], 'dropstone': ['dropstone'], 'grading': ['grading'], 'imbrication': ['imbrication'], 'intraclasts': ['intraclasts'], 'mudflakes': ['mudflakes'], 'lamination': ['lamination'], 'massive': ['massive'], 'matrix': ['matrix'], 'ripple': ['ripple'], 'deformation': ['deformation'], 'lineation': ['lineation'], 'rock': ['rock'], 'coral': ['coral'], 'regolith': ['regolith'], 'sediment': ['sediment'], 'overbank': ['overbank'], 'lag': ['lag'], 'soil': ['soil'], 'water': ['water'], 'groundwater': ['groundwater'], 'coal': ['coal'], 'vegetation': ['vegetation'], 'root': ['root'], 'alluvium': ['alluvium'], 'amphibolite': ['amphibolite'], 'artificial': ['artificial'], 'basalt': ['basalt'], 'bedrock': ['bedrock'], 'bluestone': ['bluestone'], 'cavity': ['cavity'], 'chalcedony': ['chalcedony'], 'chert': ['chert'], 'clay': ['clay'], 'claystone': ['claystone'], 'conglomerate

In [85]:
# Now we open some of the examples already given to us by Christian
# and add them to the dictionary

infile = r"C:\Users\symin\OneDrive\Documents\GA\lithology_classification\LithologyLookupTable.csv"

df_kwords_lookup = pd.read_csv(infile)


for index, row in df_kwords_lookup.iterrows():
    # Extract words and make lowercase
    kword = row['Lithology Simplified'].lower()
    ngis_word = row['NGIS Logged Lithology'].lower()
    # If the kword is in the dictionary keys then add the 
    # ngis word to the list if it doesn't exist
    if kword in lithology_kwords.keys():
        if ngis_word not in lithology_kwords[kword]:
            lithology_kwords[kword].append(ngis_word)
            
lithology_kwords



{'alluvium': ['alluvium', 'alluvails', 'alluvial', 'colluvium'],
 'amphibolite': ['amphibolite'],
 'angular': ['angular'],
 'artificial': ['artificial',
  'backfill',
  'bitumen',
  'concrete',
  'cuttings',
  'pavers',
  'road base',
  'roadfill',
  'rubble'],
 'banded': ['banded'],
 'basalt': ['basalt', 'baslt'],
 'bedrock': ['bedrock',
  'greenstone',
  'proterozoic',
  'rock',
  'rock ?',
  'rocks',
  'solidified',
  'stone',
  'stones',
  'stoney'],
 'bioturbation': ['bioturbation'],
 'blotched': ['blotched'],
 'bluestone': ['bluestone', 'blue stones', 'bluestones'],
 'breccia': ['breccia',
  'blocks',
  'broken',
  'fracture',
  'fractures',
  'ground',
  'powder',
  'shattered as'],
 'brown': ['brown'],
 'carbonate': ['carbonate'],
 'cavity': ['cavity',
  'big cavity',
  'cave',
  'cavernous',
  'cavities',
  'cavity ?'],
 'cement': ['cement'],
 'chalcedony': ['chalcedony', 'chalcedeny'],
 'chert': ['chert', 'calcite', 'chert cap', 'chirt'],
 'clast': ['clast'],
 'clay': ['clay'

In [86]:
# Now open the Bonaparte DAFWA bores where some interpretation has already been done

infile = r"C:\GA\boreholes\NT_WA_KeepRiver_OrdBon_WeaberPlain_bores_strater_mastersheet.xlsx"

df_bon = pd.read_excel(infile, sheet_name ="Lithology")

# For now we will drop nulls

df_bon=df_bon[~df_bon['LITHOLOGY_1'].isnull()]

In [87]:
# Import the drillers logs from EK

infile = r"C:\GA\boreholes\lithologies_scraped.csv"

df_ek = pd.read_csv(infile)

df_ek.drop(columns =['Unnamed: 0'], inplace=True)

# Remmove duplicate entries

df_ek = df_ek.drop_duplicates()

# For now we will drop nulls

df_ek=df_ek[~df_ek['Lithology'].isnull()]

In [88]:
# Now scrapte the NGIS database for additional lithologies

# This database is a copy of key tables for boreholes in the NGIS
# database that fall within the EFtF project areas
db = r"C:\GA\boreholes\Borehole_Data_Consolidation_CompositeLogs_NeilFork\EFTF_BH.sqlite"

import sqlite3

conn = sqlite3.connect(db)

# open the lithologies table using a sql qury in pandas
query = 'SELECT * from Lithology;'
df_ngis = pd.read_sql(query, conn)


In [89]:
# List unregistered words
unregistered_words = []

unregistered_words += findUnregisteredWords(lithology_kwords,
                                            df_bon['LITHOLOGY_1'].unique())
unregistered_words += findUnregisteredWords(lithology_kwords,
                                            df_ek['Lithology'].unique())

unregistered_words += findUnregisteredWords(lithology_kwords,
                                            df_ngis['l_Description'].unique())


In [90]:
# Create a counter to help us visualise the most common words 
cnt = Counter()
for word in unregistered_words:
    cnt[word]+=1

# Visualise the top 1000 words
cnt.most_common(1000)

[('with', 5260),
 ('some', 1659),
 ('fractured', 1018),
 ('minor', 558),
 ('fresh', 433),
 ('cemented', 372),
 ('slightly', 322),
 ('damp', 309),
 ('small', 269),
 ('sticky', 269),
 ('redbrown', 265),
 ('layers', 254),
 ('coloured', 233),
 ('pale', 216),
 ('above', 203),
 ('micaceous', 200),
 ('fragments', 191),
 ('colour', 180),
 ('caco3', 167),
 ('strength', 158),
 ('subangular', 156),
 ('fines', 154),
 ('chips', 149),
 ('from', 148),
 ('10', 146),
 ('highly', 141),
 ('trace', 140),
 ('carbonaceous', 139),
 ('various', 136),
 ('seams', 135),
 ('decomposed', 135),
 ('cream', 130),
 ('stiff', 127),
 ('greybrown', 126),
 ('interbedded', 125),
 ('subrounded', 125),
 ('nodules', 124),
 ('5', 123),
 ('struck', 121),
 ('large', 114),
 ('up', 112),
 ('20', 112),
 ('peds', 112),
 ('plastic', 109),
 ('becoming', 108),
 ('forming', 108),
 ('fracturing', 108),
 ('high', 106),
 ('50', 101),
 ('but', 95),
 ('good', 93),
 ('flecks', 93),
 ('feldspathic', 90),
 ('yellowbrown', 89),
 ('consisting', 8

In [91]:
# Here we go through the unregistered words and 
# do a fuzzy string match with keywords in the dictionary

# iterate through all words -- this may take a while
for item in unregistered_words:
    # Find the five closest string matchs
    result = process.extract(item, lithology_kwords.keys(), limit=5)
    # Iterate through these string matches
    for i in range(len(result)):
        # If the 'score' is between 100 and 85 then it is considered
        # a match
        if result[i][1] >=85.:
            # append to the dataframe 
            lithology_kwords[result[i][0]].append(item)
            
# Remove duplicates

for item in lithology_kwords.keys():
    lithology_kwords[item] = list(set(lithology_kwords[item]))

In [94]:
# Open the key word dictionary from a json file or save as a json file


filename = r"C:\Users\symin\OneDrive\Documents\GA\lithology_classification\NGIS_lithology_kwords.json"

#with open(filename, 'r') as f:
#    lithology_kwords = json.load(f)

with open(filename, 'w') as f:
    # Make sure the dictionary exists otherwise the json will be wiped
    try:
        lithology_kwords
    except NameError:
        print("Dictionary undefined")
    else:
        json.dump(lithology_kwords, f,
                  indent=4, sort_keys=True)

Next is a manual step of removing incorrect entries. A commmon error is for a match between two words that are similar but there are actually separate classes for each. For example "pinksiltstone" matches 'silt' incorrectly when it should match siltstone. Furthermore there are some spurious results like 'clay' for 'noclay' or just some errors "red" matching with "weathered".

All in all the editting job is best done with a human eye on the json file within a text editor

In [51]:
# To save the edits we make we will reimport the file and find the differences and save them into dictionaries

with open(filename, 'r') as f:
    lithology_kwords_modified = json.load(f)

    
deleted_entries = {}
added_entries = {}

for item in lithology_kwords:
    deleted_entries[item] = []
    added_entries[item] = []
    
    try:
        for entry in lithology_kwords[item]:
            if entry not in lithology_kwords_modified[item]:
                deleted_entries[item].append(entry)
    except KeyError:
        pass
    
for item in lithology_kwords_modified:
    try:
        for entry in lithology_kwords_modified[item]:
            if entry not in lithology_kwords[item]:
                added_entries[item].append(entry)
    except KeyError:
        added_entries[item] = []

In [95]:
# Save chaNGES dictionaries as a json file

dict_changes = {'added_entries': added_entries,
                'deleted_entries': deleted_entries}

filename = r"C:\Users\symin\OneDrive\Documents\GA\lithology_classification\lithology_kwords_manual_edits.json"

with open(filename, 'w') as f:
    json.dump(dict_changes, f, indent=4)

In [92]:
# To redo the editting using the saved changes run this cell

for item in [x for x in deleted_entries if x in lithology_kwords.keys()]:
    if len(deleted_entries[item]) != 0:
        for entry in deleted_entries[item]:
            try:
                lithology_kwords[item].remove(entry)
            except ValueError:
                pass

for item in added_entries:
    try:
        if len(added_entries[item]) != 0:
            for entry in added_entries[item]:
                if entry not in lithology_kwords[item]: 
                    lithology_kwords[item].append(entry)
    except ValueError:
        lithology_kwords[item] = added_entries[item]           

In [103]:
# Here is an example of how this data could be used to create a bag of words for an ngis bore

# First we select a bore from within the Ord-Keep project area - uid extracted from GIS


df_ek_lith = df_ngis[df_ngis['uid'] == 2869]['l_Description']
print(df_ek_lith)

18597    Sand: f - med gn; loose; clear - wh gns; loose...
18598    50% sand: f gned; milky to clear gns; poorly s...
18599                                  Clay: plastic clay.
18600                                  Clay: plastic clay.
18601                Clay: red bwn; pebbles of rock frags.
18602    Sand: med gned; some pebble; red bwn soft clay...
18603    Gravel: red bwn; 60% rock frag; rock frags; ve...
18604    50% sand: v fgn with clay matrix - red with qu...
18605    50% sand: v fgn with clay matrix - red with qu...
18606    Sand: med gn; loose; orange col; subrded; poor...
18607    Sand: f - med gn; milky wh to or - wh; qtz gn;...
18608    Sand: f - med gn; milky wh to or - wh; qtz gn;...
Name: l_Description, dtype: object


In [134]:
# We will create a dataframe to display the results

df_results = pd.DataFrame(columns = lithology_kwords.keys())

# Iterate through the panda series
for index, string in df_ek_lith.items():
    # Get a dictionary with the word count for each keyword
    newDict = bag_of_words(string, lithology_kwords)
    # Append this to a dataframe with the same index as
    # the original data
    df_results = df_results.append(pd.DataFrame(newDict,
                                   index = [index]))

In [138]:
# We will display the results for the four most important
# sediment types
df_results[['sand','clay', 'gravel', 'silt']]

,sand,clay,gravel,silt
18597,1,1,0,0
18598,1,1,0,0
18599,0,1,0,0
18600,0,1,0,0
18601,0,1,1,0
18602,1,1,1,0
18603,0,0,1,0
18604,1,1,1,0
18605,1,1,1,0
18606,1,0,0,0
